# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
filepath = song_files[0]
filepath

'/home/gui/Projects/de-01-data-modeling-with-postgres/data/song_data/A/B/B/TRABBNP128F932546F.json'

In [7]:
df = pd.read_json(filepath, typ='series')
df

num_songs                            1
artist_id           AR62SOJ1187FB47BB5
artist_latitude                   None
artist_longitude                  None
artist_location                       
artist_name             Chase & Status
song_id             SOGVQGJ12AB017F169
title                        Ten Tonne
duration                     337.68444
year                              2005
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values
song_data

array(['SOGVQGJ12AB017F169', 'Ten Tonne', 'AR62SOJ1187FB47BB5', 2005,
       337.68444], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
print(f"song_table_insert = {song_table_insert}")
print(f"song_data = {song_data}")
cur.execute(song_table_insert, song_data)
conn.commit()

song_table_insert = 
    INSERT INTO songs (
        song_id,
        title,
        artist_id,
        year,
        duration
    )
    VALUES (%s, %s, %s, %s, %s)

song_data = ['SOGVQGJ12AB017F169' 'Ten Tonne' 'AR62SOJ1187FB47BB5' 2005 337.68444]


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
df.index

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [11]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values
artist_data

array(['AR62SOJ1187FB47BB5', 'Chase & Status', '', None, None],
      dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data/')

In [14]:
filepath = log_files[0]

In [15]:
df = pd.read_json(filepath, lines=True)
print(f"df.columns = {df.columns}")
df.head()

df.columns = Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Hoobastank,Logged In,Cierra,F,0,Finley,241.39710,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Say The Same,200,1541808927796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
1,Mark Knopfler,Logged In,Cierra,F,1,Finley,249.31220,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Why Aye Man,200,1541809168796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
2,Mogwai,Logged In,Cierra,F,2,Finley,341.28934,free,"Richmond, VA",PUT,NextSong,1541013292796,132,We're No Here,200,1541809417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
3,The Casualties,Logged In,Cierra,F,3,Finley,181.49832,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Punx Unite,200,1541809758796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
4,None,Logged In,Cecilia,F,0,Owens,NaN,free,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1541032432796,424,None,200,1541813635796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,6


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Hoobastank,Logged In,Cierra,F,0,Finley,241.39710,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Say The Same,200,1541808927796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
1,Mark Knopfler,Logged In,Cierra,F,1,Finley,249.31220,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Why Aye Man,200,1541809168796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
2,Mogwai,Logged In,Cierra,F,2,Finley,341.28934,free,"Richmond, VA",PUT,NextSong,1541013292796,132,We're No Here,200,1541809417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
3,The Casualties,Logged In,Cierra,F,3,Finley,181.49832,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Punx Unite,200,1541809758796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
5,The Living End,Logged In,Ryan,M,0,Smith,188.62975,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,433,Roll On (Album Version),200,1541822502796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [17]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday 
# from the `ts` column and set `time_data` to a list containing these values in order


t = pd.to_datetime(df['ts'])
t.head()

0   1970-01-01 00:25:41.808927796
1   1970-01-01 00:25:41.809168796
2   1970-01-01 00:25:41.809417796
3   1970-01-01 00:25:41.809758796
5   1970-01-01 00:25:41.822502796
Name: ts, dtype: datetime64[ns]

In [18]:
time_data = (
    t.values,
    t.dt.hour.values,
    t.dt.day.values,
    t.dt.weekofyear.values,
    t.dt.month.values,
    t.dt.year.values,
    t.dt.weekday.values
)
column_labels = (
    'start_time',
    'hour',
    'day',
    'week',
    'month',
    'year',
    'weekday'
)

/tmp/ipykernel_262013/1359514666.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  t.dt.weekofyear.values,


In [19]:
time_df = pd.DataFrame(data=time_data).T
time_df.columns=column_labels
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1970-01-01 00:25:41.808927796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.809168796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.809417796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.809758796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.822502796,0,1,1,1,1970,3


In [20]:
time_df = time_df.astype({
    'hour': int,
    'day': int,
    'week': int,
    'month': int,
    'year': int,
    'weekday': int,
})

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
# FIXME timestamp insert in postgres

for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.drop_duplicates(inplace=True)
user_df.head()

#TODO drop duplicates?

/tmp/ipykernel_262013/1065050003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_df.drop_duplicates(inplace=True)


,userId,firstName,lastName,gender,level
0,96,Cierra,Finley,F,free
5,26,Ryan,Smith,M,free
6,16,Rylan,George,M,free
8,44,Aleena,Kirby,F,paid
45,37,Jordan,Hicks,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
df[[
    'ts',
    'userId',
    'level',
    'song',
    'artist',
    'length',
    'sessionId',
    'location',
    'userAgent',

]].head()


,ts,userId,level,song,artist,length,sessionId,location,userAgent
0,1541808927796,96,free,Say The Same,Hoobastank,241.39710,132,"Richmond, VA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
1,1541809168796,96,free,Why Aye Man,Mark Knopfler,249.31220,132,"Richmond, VA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,1541809417796,96,free,We're No Here,Mogwai,341.28934,132,"Richmond, VA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,1541809758796,96,free,Punx Unite,The Casualties,181.49832,132,"Richmond, VA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
5,1541822502796,26,free,Roll On (Album Version),The Living End,188.62975,433,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."


In [25]:
df['ts'] = pd.to_datetime(df['ts'])

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    
    results = cur.fetchone()

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (
        index,
        row['ts'],
        row['userId'],
        row['level'],
        songid,
        artistid,
        row['sessionId'],
        row['location'],
        row['userAgent']
    )
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

cur.query = b"\n    SELECT song_id, songs.artist_id FROM songs\n    JOIN artists ON songs.artist_id = artists.artist_id\n    WHERE title='Say The Same' AND artist_name='Hoobastank' AND duration=241.3971;\n"
cur.query = b"\n    SELECT song_id, songs.artist_id FROM songs\n    JOIN artists ON songs.artist_id = artists.artist_id\n    WHERE title='Why Aye Man' AND artist_name='Mark Knopfler' AND duration=249.3122;\n"
cur.query = b"\n    SELECT song_id, songs.artist_id FROM songs\n    JOIN artists ON songs.artist_id = artists.artist_id\n    WHERE title='We''re No Here' AND artist_name='Mogwai' AND duration=341.28934;\n"
cur.query = b"\n    SELECT song_id, songs.artist_id FROM songs\n    JOIN artists ON songs.artist_id = artists.artist_id\n    WHERE title='Punx Unite' AND artist_name='The Casualties' AND duration=181.49832;\n"
cur.query = b"\n    SELECT song_id, songs.artist_id FROM songs\n    JOIN artists ON songs.artist_id = artists.artist_id\n    WHERE title='Roll On (Album Version)' AND a

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [26]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.